In [ ]:
# some notes:

# one gender per paper

In [97]:
import pandas as pd
import numpy as np
from ast import literal_eval
import math
import pickle
import json

import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

import geopandas
import plotly.express as px

In [2]:
data = pd.read_csv('../ete3/Plt_sci_publications_geo_species_8.26.csv', low_memory=False)

In [3]:
# many of these are one author with multiple affiliations

mult_corresp = []

for i in range(len(data['Corresponding author Addresses'])):
    if ';' in data.loc[i]['Corresponding author Addresses']:
        mult_corresp.append(i)

In [4]:
data['Corresponding author geocoords'] = [i.split('; ') for i in data['Corresponding author geocoords']]

In [5]:
data['map_props'] = [1 for i in range(len(data))]

In [6]:
out = data.explode('Corresponding author geocoords')
out['map_props'] /= out['map_props'].groupby(level=0).transform('count')

In [7]:
out['old_index'] = out.index

In [8]:
out = out.reset_index(drop=True)

In [9]:
# do this after exploding since exploding needs to be on strings
out['Corresponding author geocoords'] = [literal_eval(x) for x in out['Corresponding author geocoords']]

In [10]:
out

,Author Full Names,Article Title,Journal,impact factor (2020),Author Keywords,Keywords Plus,Abstract,Addresses--all authors,Nations--all authors,Corresponding author Addresses,...,_merge,Locations--all authors,Geocoords--all authors,Corresponding author locations,Corresponding author geocoords,Number of authoring countries,Number of corresponding countries,unique_NCBI,map_props,old_index
0,"Raho, Nicolas; Fraga, Santiago; Abad, Jose P.;...","Biecheleria tirezensis sp nov (Dinophyceae, Su...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Athalassic; CBC; dinoflagellate; halotolerant;...,INTERNAL TRANSCRIBED SPACER; RIBOSOMAL-RNA; LA...,A new euryhaline and eurythermal dinoflagellat...,"[Raho, Nicolas; Abad, Jose P.; Marin, Irma] Un...",Spain; United States,"Abad, JP; Marin, I (corresponding author), Uni...",...,both,"Madrid, Mostoles, Spain; Vigo, Pontevedra, Spa...","(40.3232129, -3.8676291); (42.24059889999999, ...","Madrid, Mostoles, Spain","(40.3232129, -3.8676291)",2,1,"[1209074, 1523459]",1.0,0
1,"Abdel-Hafez, Sobhy I. I.; Abo-Elyousr, Kamal A...",Fungicidal activity of extracellular products ...,EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Alternaria porri; cyanobacterial extracellular...,BLUE-GREEN; ANTIFUNGAL ACTIVITY; PURPLE BLOTCH...,Although cyanobacteria are recognized as renew...,"[Abdel-Hafez, Sobhy I. I.; Abdel-Rahim, Ismail...",Egypt,"Abdel-Rahim, IR (corresponding author), Assiut...",...,both,"Assiut, Egypt","(27.1783117, 31.1859257)","Assiut, Egypt","(27.1783117, 31.1859257)",1,1,"[48098, 1159, 1841509, 418409, 1117, 35824]",1.0,1
2,"Abdelahad, Nadia; Bolpagni, Rossano; Jona Lasi...","Distribution, morphology and ecological niche ...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Autecology; freshwater red algae; Italy; lowla...,RED ALGAE; HELMINTHOSUM BATRACHOSPERMALES; SEC...,Twenty-three freshwater springs of the Po plai...,"[Abdelahad, Nadia; Amadio, Carolina] Sapienza ...",Italy; United States,"Abdelahad, N (corresponding author), Sapienza ...",...,both,"Rome, RM, Italy; Sez Istol, Parma, Italy; Athe...","(39.3292396, -82.1012554); (41.90278350000001,...","Rome, RM, Italy","(41.90278350000001, 12.4963655)",2,1,"[340433, 2809, 1486645, 69142]",1.0,2
3,"Aboal, Marina; Chapuis, Iara; Paiano, Monica O...","Diversity of Chroothece (Rhodophyta, Stylonema...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Chroothece; cryptic diversity; ecology; phylog...,RED ALGAE RHODOPHYTA; RICHTERIANA RHODOPHYTA; ...,Chroothece has been reported from a range of f...,"[Aboal, Marina] Fac Biol, Dept Plant Biol, Lab...",Spain; Australia; England; Brazil,"Aboal, M (corresponding author), Fac Biol, Dep...",...,both,"Murcia, Espinardo, Spain; Armilla, Granada, Sp...","(-20.8127115, -49.376521); (-37.7787036, 144.9...","Murcia, Espinardo, Spain","(38.0111896, -1.1556962)",4,1,"[2115879, 2115880, 362227]",1.0,3
4,"Aboal, Marina; Eugenia Garcia-Fernandez, Maria...","Ecology, morphology and physiology of Chroothe...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,calcareous Chroothece; colony; lipid; organic ...,DIATOM DIDYMOSPHENIA; ALGAL COMMUNITIES; NITRO...,The ecology of Chroothece was studied in the h...,"[Aboal, Marina; Eugenia Garcia-Fernandez, Mari...",Spain; England,"Aboal, M (corresponding author), Univ Murcia, ...",...,both,"Murcia, Espinardo, Spain; Bellaterra, Cataloni...","(38.0111896, -1.1556962); (41.5037176, 2.08640...","Murcia, Espinardo, Spain","(38.0111896, -1.1556962)",2,1,"[362227, 373984]",1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314164,"Senoussi, Mohammed Mourad; Nora, Bassa; Joel, ...",Impact of hypoxia on the growth and alkaloid a...,ACTA PHYSIOLOGIAE PLANTARUM,2.354,Catharanthus roseus; Ajmalicine; Hypoxia; Cyto...,HIGH-DENSITY CULTURES; AJMALICINE PRODUCTION; ...,The Madagascar periwinkle (Catharanthus roseus...,"[Senoussi, Mohammed Mourad; Nora, Bassa] Univ ...",Algeria; France,"Senoussi, MM (corresponding author), Univ Larb...",...,right_only,"Oum El Bouaghi, Algeria; Tours, France","(35.8688789, 7.110

In [14]:
out = out.join(out['Corresponding author geocoords'].apply(pd.Series)).rename(columns={0:'Latitude', 1:'Longitude'})

In [15]:
out

,Author Full Names,Article Title,Journal,impact factor (2020),Author Keywords,Keywords Plus,Abstract,Addresses--all authors,Nations--all authors,Corresponding author Addresses,...,Geocoords--all authors,Corresponding author locations,Corresponding author geocoords,Number of authoring countries,Number of corresponding countries,unique_NCBI,map_props,old_index,Latitude,Longitude
0,"Raho, Nicolas; Fraga, Santiago; Abad, Jose P.;...","Biecheleria tirezensis sp nov (Dinophyceae, Su...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Athalassic; CBC; dinoflagellate; halotolerant;...,INTERNAL TRANSCRIBED SPACER; RIBOSOMAL-RNA; LA...,A new euryhaline and eurythermal dinoflagellat...,"[Raho, Nicolas; Abad, Jose P.; Marin, Irma] Un...",Spain; United States,"Abad, JP; Marin, I (corresponding author), Uni...",...,"(40.3232129, -3.8676291); (42.24059889999999, ...","Madrid, Mostoles, Spain","(40.3232129, -3.8676291)",2,1,"[1209074, 1523459]",1.0,0,40.323213,-3.867629
1,"Abdel-Hafez, Sobhy I. I.; Abo-Elyousr, Kamal A...",Fungicidal activity of extracellular products ...,EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Alternaria porri; cyanobacterial extracellular...,BLUE-GREEN; ANTIFUNGAL ACTIVITY; PURPLE BLOTCH...,Although cyanobacteria are recognized as renew...,"[Abdel-Hafez, Sobhy I. I.; Abdel-Rahim, Ismail...",Egypt,"Abdel-Rahim, IR (corresponding author), Assiut...",...,"(27.1783117, 31.1859257)","Assiut, Egypt","(27.1783117, 31.1859257)",1,1,"[48098, 1159, 1841509, 418409, 1117, 35824]",1.0,1,27.178312,31.185926
2,"Abdelahad, Nadia; Bolpagni, Rossano; Jona Lasi...","Distribution, morphology and ecological niche ...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Autecology; freshwater red algae; Italy; lowla...,RED ALGAE; HELMINTHOSUM BATRACHOSPERMALES; SEC...,Twenty-three freshwater springs of the Po plai...,"[Abdelahad, Nadia; Amadio, Carolina] Sapienza ...",Italy; United States,"Abdelahad, N (corresponding author), Sapienza ...",...,"(39.3292396, -82.1012554); (41.90278350000001,...","Rome, RM, Italy","(41.90278350000001, 12.4963655)",2,1,"[340433, 2809, 1486645, 69142]",1.0,2,41.902784,12.496365
3,"Aboal, Marina; Chapuis, Iara; Paiano, Monica O...","Diversity of Chroothece (Rhodophyta, Stylonema...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Chroothece; cryptic diversity; ecology; phylog...,RED ALGAE RHODOPHYTA; RICHTERIANA RHODOPHYTA; ...,Chroothece has been reported from a range of f...,"[Aboal, Marina] Fac Biol, Dept Plant Biol, Lab...",Spain; Australia; England; Brazil,"Aboal, M (corresponding author), Fac Biol, Dep...",...,"(-20.8127115, -49.376521); (-37.7787036, 144.9...","Murcia, Espinardo, Spain","(38.0111896, -1.1556962)",4,1,"[2115879, 2115880, 362227]",1.0,3,38.011190,-1.155696
4,"Aboal, Marina; Eugenia Garcia-Fernandez, Maria...","Ecology, morphology and physiology of Chroothe...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,calcareous Chroothece; colony; lipid; organic ...,DIATOM DIDYMOSPHENIA; ALGAL COMMUNITIES; NITRO...,The ecology of Chroothece was studied in the h...,"[Aboal, Marina; Eugenia Garcia-Fernandez, Mari...",Spain; England,"Aboal, M (corresponding author), Univ Murcia, ...",...,"(38.0111896, -1.1556962); (41.5037176, 2.08640...","Murcia, Espinardo, Spain","(38.0111896, -1.1556962)",2,1,"[362227, 373984]",1.0,4,38.011190,-1.155696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314164,"Senoussi, Mohammed Mourad; Nora, Bassa; Joel, ...",Impact of hypoxia on the growth and alkaloid a...,ACTA PHYSIOLOGIAE PLANTARUM,2.354,Catharanthus roseus; Ajmalicine; Hypoxia; Cyto...,HIGH-DENSITY CULTURES; AJMALICINE PRODUCTION; ...,The Madagascar periwinkle (Catharanthus roseus...,"[Senoussi, Mohammed Mourad; Nora, Bassa] Univ ...",Algeria; France,"Senoussi, MM (corresponding author), Univ Larb...",...,"(35.8688789, 7.1108265999999976); (47.394144, ...","Oum El Bouaghi, Algeria","(35.8688789, 7.1108265999999976)",2,1,[4058],1.0,296442,35.868879,7.110827
314165,"Boutoumi, Hocine; Moulay, Saad; Khodja, Mohamed",Essential Oil from R

In [11]:
# these are polygons representing states and provinces
shpfilename = shapereader.natural_earth('10m', 'cultural', 'admin_1_states_provinces')
# shpfilename2 = shapereader.natural_earth('10m', 'cultural', 'admin_1_states_provinces_lines')

In [12]:
df2 = geopandas.read_file(shpfilename)

In [13]:
df2

,featurecla,scalerank,adm1_code,diss_me,iso_3166_2,wikipedia,iso_a2,adm0_sr,name,name_alt,...,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,FCLASS_TLC,geometry
0,Admin-1 states provinces,3,ARG-1309,1309,AR-E,None,AR,1,Entre RÃ­os,Entre-Rios,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-58.20011 -32.44713, -58.20012 -32.4..."
1,Admin-1 states provinces,6,URY-8,8,UY-PA,None,UY,1,PaysandÃº,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-58.20012 -32.44720, -58.20011 -32.4..."
2,Admin-1 states provinces,2,IDN-1185,1185,ID-KI,None,ID,5,Kalimantan Timur,Kaltim,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
3,Admin-1 states provinces,6,MYS-1186,1186,MY-12,None,MY,5,Sabah,North Borneo,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
4,Admin-1 states provinces,3,CHL-2694,2694,CL-AP,None,CL,1,Arica y Parinacota,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-69.51009 -17.50659, -69.50611 -17.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4591,Admin-1 states provinces,10,MLT-5953,5953,MT-20,None,MT,3,Isla,Isla|CittÃ Invicta,...,None,None,None,None,None,None,None,None,None,"POLYGON ((14.51073 35.88896, 14.51537 35.89113..."
4592,Admin-1 states provinces,10,MLT-5955,5955,MT-06,None,MT,3,Cospicua,Bormla,...,None,None,None,None,None,None,None,None,None,"POLYGON ((14.52937 35.88156, 14.52700 35.87772..."
4593,Admin-1 states provinces,10,MLT-5942,5942,MT-41,None,MT,3,PietÃ,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((14.49801 35.89251, 14.49616 35.88830..."
4594,Admin-1 states provinces,10,MLT-5940,5940,MT-34,None,MT,3,Msida,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((14.48663 35.88636, 14.48231 35.88979..."


In [16]:
gdf = geopandas.GeoDataFrame(
    out, geometry=geopandas.points_from_xy(out.Longitude, out.Latitude))

In [76]:
# about 2.5k missing locations
gdf_out = geopandas.sjoin(gdf, df2[['diss_me', 'geometry']], how='left', op='within')

/home/sarah/anaconda3/envs/nlp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [85]:
gdf_out = gdf_out.dropna(axis=0, subset='diss_me')

In [89]:
gdf_out['diss_me'] = [int(i) for i in gdf_out['diss_me']]

/home/sarah/anaconda3/envs/nlp/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [108]:
gdf_out[['map_props','diss_me','inferred_gender']]

,map_props,diss_me,inferred_gender
0,1.0,5833,male
1,1.0,1549,male
2,1.0,5422,female
3,1.0,5836,female
4,1.0,5836,female
...,...,...,...
314164,1.0,2221,male
314165,1.0,2209,male
314166,1.0,2217,male
314167,1.0,2195,NaN


In [91]:
final_numbers = gdf_out[['map_props', 'diss_me']].groupby(by=['diss_me']).sum()

In [94]:
final_numbers = final_numbers.reset_index()

In [95]:
final_numbers

,diss_me,map_props
0,2,15.500000
1,3,58.000000
2,4,5.500000
3,8,11.000000
4,10,8.000000
...,...,...
1729,10372,371.833333
1730,10373,5.000000
1731,20010,40.000000
1732,20011,739.666667


In [100]:
# Opening JSON file
f = open('admin1.geojson')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Closing file
f.close()

In [109]:
# fig = px.choropleth(final_numbers, geojson=data, color="map_props",
#                     locations="diss_me", featureidkey="properties.id",
#                     projection="robinson"
#                    )
# fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# fig.update_traces(marker_line_width=0)

# fig.show()
# fig.write_html("map.html")

In [ ]:
np.where(len(data['Corresponding author geocoords'])>2)

In [32]:
with open('region.pickle', 'rb') as handle:
    region = pickle.load(handle)